In [1]:
import requests
import pandas as pd

from datetime import datetime

from openai import OpenAI
from bs4 import BeautifulSoup

client = OpenAI(
    api_key="" # PUT YOUR API KEY here
)

In [11]:
# Lamine Yamal
player_name = 'Lamine Yamal'
url = 'https://fbref.com/en/players/82ec26c1/Lamine-Yamal'
attrs = 'scout_summary_AM'

# # Nathan Broadhead
player_name = 'Nathan Broadhead'
url = 'https://fbref.com/en/players/43309491/Nathan-Broadhead'
attrs = 'scout_summary_AM'


df = pd.read_html(
    url, 
    attrs={'id': attrs} # Change the id & class to correspond to the table
)[0]

In [12]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [13]:
# Get the player's age, team, league, and position
position = soup.select_one('p:-soup-contains("Position")').text.split(':')[-2].split(' ')[0].strip()
birthday = soup.select_one('span[id="necro-birth"]').text.strip()

# get the player's age
age = (datetime.now() - datetime.strptime(birthday, '%B %d, %Y')).days // 365
team = soup.select_one('p:-soup-contains("Club")').text.split(':')[-1].strip()

In [14]:
# df.columns = df.columns.droplevel(0)

df = df.dropna(subset='Statistic')

In [15]:
prompt = f"""

I need you to create a scouting report on {player_name}. Can you provide me with a summary of their strengths and weaknesses?

Here is the data I have on him:

Player: {player_name}
Position: {position}
Age: {age}
Team: {team}

{df.to_markdown()}


Return the scouting report in the following markdown format:

# Scouting Report for {player_name}

## Strengths
< a list of 1 to 3 strengths >

## Weaknesses
< a list of 1 to 3 weaknesses >

## Summary
< a brief summary of the player's overall performance and if he would be beneficial to the team >
"""

In [16]:
print(prompt)



I need you to create a scouting report on Nathan Broadhead. Can you provide me with a summary of their strengths and weaknesses?

Here is the data I have on him:

Player: Nathan Broadhead
Position: FW-MF (AM)
Age: 26
Team: Ipswich Town

|    | Statistic                | Per 90   |   Percentile |
|---:|:-------------------------|:---------|-------------:|
|  0 | Non-Penalty Goals        | 0.51     |           98 |
|  1 | npxG: Non-Penalty xG     | 0.35     |           94 |
|  2 | Shots Total              | 2.94     |           87 |
|  3 | Assists                  | 0.12     |           28 |
|  4 | xAG: Exp. Assisted Goals | 0.22     |           72 |
|  5 | npxG + xAG               | 0.57     |           88 |
|  6 | Shot-Creating Actions    | 3.10     |           31 |
|  8 | Passes Attempted         | 26.29    |           14 |
|  9 | Pass Completion %        | 75.2%    |           60 |
| 10 | Progressive Passes       | 3.14     |           36 |
| 11 | Progressive Carries      | 2.35   

In [17]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": "You are a professional football (soccer) scout."},
        {"role": "user", "content": prompt},
    ],
)

In [18]:
print(response.choices[0].message.content)

# Scouting Report for Nathan Broadhead

## Strengths
- **Scoring Ability:** Nathan stands out in his ability to score non-penalty goals, with a per 90 statistic of 0.51, placing him in the 98th percentile. His non-penalty expected goals (npxG) is also high at 0.35 per 90, ranked in the 94th percentile, which suggests that his high scoring rate is sustainable and not based on luck.
- **Shot Volume:** He demonstrates a notable propensity to take shots, averaging 2.94 shots per 90, positioning him in the 87th percentile among peers. This trait indicates his confidence and willingness to create scoring opportunities.
- **Defensive Contributions:** Broadhead shows an above-average ability to contribute defensively for an attacking player, evidenced by his blocks per 90 minutes at 1.37, ranking him in the 90th percentile.

## Weaknesses
- **Creating for Others:** Despite being effective in front of goal, Broadhead shows limitations in his playmaking, with assists at 0.12 per 90 (28th percent

In [19]:
# save the scouting report to a new markdown file
with open(f"/Users/mckayjohns/Documents/GitHub/youtube-videos/{player_name}_scouting_report.md", "w") as f:
    f.write(response.choices[0].message.content)